## Out methods
1) Try raw jammin scores
2) Try normalized jammin scores
3) Normalized jammin + vader scores 
4) Balance dataset

In [8]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import requests
import json

from random import shuffle

import numpy as np

import time

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn import preprocessing



from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline


In [9]:
def jammin_emotion(text):
    # Pre process text
    text = text.replace('\\"','"')
    text = text.replace('"','\\"')
    text = text.replace("\n"," ")
    text = text.replace('â€™',"'")
    text = text.replace('â€œ','\\"')
    text = text.replace('â€','\\"')
    text = text.replace('â€”','-')
    text = text.replace('â€¦','...')
    text = text.replace('ðŸ¤”',' ')
    text = text.replace('Ÿ™„',' ')
    text = text.replace('â€˜',"'")
    text = text.replace('ðŸ‘',' ')
    text = text.replace('ðŸ˜‰',' ')
    text = text.replace('ðŸ¤¨',' ')
    text = text.replace('ðŸ˜„',' ')
    text = text.replace('Ÿ˜‚',' ')
    text = text.replace('Ÿ˜',' ')
    text = text.replace('€‹',' ')
    text = text.replace('€‹',' ')
    text = text.replace('„',' ')
    text = text.replace('Ÿ',' ')
    text = text.replace('€',' ')
    text = text.replace('™‚',' ')
    text = text.replace('’',' ')
    text = text.replace('™',' ')
    text = text.replace('‡',' ')
    text = text.replace('‘Š',' ')
    text = text.replace('–',' ')
    text = text.replace('“',' ')
    text = text.replace('”',' ')
    text = text.replace('š',' ')
    text = text.replace('˜',' ')
    text = text.replace('œ…',' ')
    text = text.replace('œ',' ')
    text = text.replace('‹',' ')
    text = text.replace('Œ',' ')
    text = text.replace('›',' ')
    text = text.replace('‘',' ')
    text = text.replace('…',' ')
    text = text.replace('Ž',' ')
    
    print(text)
 
    payload = '{"text":"%s","lang":"en"}'%(text)
    #print payload
    headers = {'content-Type': 'application/json'}

    url = 'http://1.34.96.63:8080/webresources/jammin/emotion'
    #url = 'http://127.0.0.1:8080/JamminTextEmotionAPI/webresources/jammin/emotion'

    r = requests.post(url,  data=payload, headers=headers)
    #print(r)
    return r.json()

def contains_a_d_or_b(response):
    if "text" in response:
        if "yes" in response["bullying"]:
            #print "Contains bullying"
            return True
        for group in response["groups"]:
            if "anger" in group["name"]:
                #print "Contains anger"
                return True

            if "disgust" in group["name"]:
                #print "Contains disgust"
                return True
    return False


def get_negativity_ind_posts(post,snr,  an, ac):
    vector = []

    # Count the number of negative sentences
    count_neg_sents = 0
    neg_ratio = 0.0
    count_bullying = 0
    bullying_ratio = 0.0

    negs = []
    compounds = []

    # To get the average sentiment scores
    tot_sentences = len(post["sentences"])
    
    # To determine if this comment contains negativity
    neg_comment = False
    
    for sent in post["sentences"]:

        # Get the Vader scores
        vs = sent["vader"]
        neg = float(vs["neg"])
        compound = float(vs["compound"])

        negs.append(neg)
        compounds.append(compound)
        # Add the negativity and compound
        #sum_neg = sum_neg + neg
        #sum_compound = sum_compound + compound

        # Get the Jammin scores
        je = sent["jammin"]

        # If both jammin and vader detect negativity
        if contains_a_d_or_b(je) and (neg >= 0.05 or compound <= -0.05):
            # Mark this comment as being negative
            neg_comment = True

            count_neg_sents = count_neg_sents + 1
            if "yes" in je["bullying"]:
                count_bullying = count_bullying + 1

    # Flag the post a negative
    if neg_comment:
        vector.append(1.0)
    else:
        vector.append(0.0)
        

    #avg_neg = float(sum_neg) / float(tot_sentences)
    #avg_compound = float(sum_compound) / float(tot_sentences)

    avg_neg = np.mean(negs)
    avg_compound = np.mean(compounds)
    
    vector.append(avg_neg)
    vector.append(avg_compound)

    #if tot_sentences > 0:
    # Percentage of neg comments for this post
    #neg_ratio = float(count_neg) / float(tot_sentences)
    # Percentage of neg sentences for this post
    sent_neg_ratio = float(count_neg_sents) / float(tot_sentences)
    # Percentage of bullying detectec on this post
    bullying_ratio = float(count_bullying) / float(tot_sentences)
    
    vector.append(sent_neg_ratio)
    vector.append(bullying_ratio)

    if sent_neg_ratio >= snr and (avg_neg >= an or avg_compound <= -ac):
        # Determine the level
        level = 1

    return vector


In [10]:
sentence = "Excellent reminder of where things could go! California and Sanctuary cities do not speak for the rest of Americans anymore than the KKK and racial laws during the Civil Rights movements.and the illegal laws that accompanied them! Illegal immigrants do not belong here, they are illegal. If immigrants want protection, and then do what you need to become legal and then I will welcome them!"
sentence = ' " I love how they are so "concerned" about gun control.'

In [11]:
print (jammin_emotion(sentence))

 \" I love how they are so \"concerned\" about gun control.
{'groups': [{'emotions': ['bullying', 'aggravated', 'furious', 'pissedoff', 'irritated'], 'name': 'anger'}, {'emotions': ['conflicted', 'hopeless', 'awful'], 'name': 'sadness'}], 'text': '" i love how they are so "concerned" about gun control.', 'lang': 'en', 'ambiguous': 'no', 'bullying': 'yes'}


In [12]:
analyzer = SentimentIntensityAnalyzer()

In [13]:
vs = analyzer.polarity_scores(sentence)
print (vs)

{'neg': 0.192, 'neu': 0.505, 'pos': 0.303, 'compound': 0.3687}


In [ ]:
json_samples = []

with open('../twitter-hatespeech/data/data_manual.json') as json_file:  
#with open('../Classifier/data_manual_axel_plus_turk.json') as json_file:
    data = json.load(json_file)
    
    
    
    for i, p in enumerate(data):
        text = p['text']
        
        
        #if i < 53:
        #    continue
            
        if i == 1197:
            continue
        #if i > 10:
        #    break
        
        print(i)
        # List to store the sentences and their emotions
        sentences = []
        
        # Split into sentences
        tokenized_sentences = nltk.sent_tokenize(text)
        
        found = False
        for sentence in tokenized_sentences:
            sentence = sentence.strip()
            # Get Vader sentiment
            #sentence = sentence.encode('utf-8')
            #sentence = bytes(sentence, 'utf-8').decode('utf-8', 'ignore')
            print (sentence)
            #print()
            
            res = jammin_emotion(sentence)
            
            vs = analyzer.polarity_scores(sentence)
            
            
            # Add the sentence data to the list
            sent_data = {"text":sentence,"vader":vs,"jammin":res}
            #sent_data = {"text":sentence,"vader":vs}
            sentences.append(sent_data)
            #count = count + 1
        
        #print (sentences)
        p["sentences"] = sentences
        json_samples.append(p)
        
        #print('')

with open("data_manual_sentiment.json","w") as outfile:  
    json.dump(json_samples, outfile)
    outfile.close()

0
How about the illegal female immigrant from California, who got a law license and was given a state job.
How about the illegal female immigrant from California, who got a law license and was given a state job.
She is not even a citizen of the United States, only in California would this crap happen!
She is not even a citizen of the United States, only in California would this crap happen!
How is this even possible?
How is this even possible?
1
Why does it make a difference if heâ€™s an illegal immigrant or not in this situation ?
Why does it make a difference if he's an illegal immigrant or not in this situation ?
Besides fanning the flames thereâ€™s no reason for it
Besides fanning the flames there's no reason for it
2
Illegal ALIEN not illegal immigrant!
Illegal ALIEN not illegal immigrant!
Is a drug dealer an unlicensed pharmacist?
Is a drug dealer an unlicensed pharmacist?
No.
No.
Then this guys is an illegal alien.
Then this guys is an illegal alien.
Let's use the correct terms 

## Determine the emotions to include as part of the binary vector

In [19]:
f = open("features.txt", "r")
features = {}
for line in f:
    line = line.strip()
    if not "#" in line and len(line) > 0:
        features[line] = line



emotion_count = features
for emotion in emotion_count:
    emotion_count[emotion] = 0
print(emotion_count)
print(len(emotion_count))

{'aggravated': 0, 'aggressiveness': 0, 'anger': 0, 'anger2': 0, 'angry': 0, 'annoyance': 0, 'annoyed': 0, 'bummed': 0, 'enraged': 0, 'fedup': 0, 'frustrated': 0, 'furious': 0, 'grumpy': 0, 'ill': 0, 'insulted': 0, 'irritated': 0, 'mad': 0, 'offended': 0, 'pissed': 0, 'pissedoff': 0, 'rage': 0, 'bullying': 0, 'bitter': 0, 'blah': 0, 'bored': 0, 'boredom': 0, 'cheated': 0, 'contempt': 0, 'defeated': 0, 'disgust': 0, 'disgusted': 0, 'dumb': 0, 'gross': 0, 'horrible': 0, 'hungover': 0, 'lame': 0, 'lousy': 0, 'meh': 0, 'nauseous': 0, 'numb': 0, 'regret': 0, 'remorse': 0, 'rough': 0, 'shame': 0, 'stupid': 0, 'terrible': 0, 'awful': 0, 'ugly': 0, 'yucky': 0}
49


In [20]:
print(emotion_count)

{'aggravated': 0, 'aggressiveness': 0, 'anger': 0, 'anger2': 0, 'angry': 0, 'annoyance': 0, 'annoyed': 0, 'bummed': 0, 'enraged': 0, 'fedup': 0, 'frustrated': 0, 'furious': 0, 'grumpy': 0, 'ill': 0, 'insulted': 0, 'irritated': 0, 'mad': 0, 'offended': 0, 'pissed': 0, 'pissedoff': 0, 'rage': 0, 'bullying': 0, 'bitter': 0, 'blah': 0, 'bored': 0, 'boredom': 0, 'cheated': 0, 'contempt': 0, 'defeated': 0, 'disgust': 0, 'disgusted': 0, 'dumb': 0, 'gross': 0, 'horrible': 0, 'hungover': 0, 'lame': 0, 'lousy': 0, 'meh': 0, 'nauseous': 0, 'numb': 0, 'regret': 0, 'remorse': 0, 'rough': 0, 'shame': 0, 'stupid': 0, 'terrible': 0, 'awful': 0, 'ugly': 0, 'yucky': 0}


In [60]:
def all_data_to_vector(output, sentiment = True, thresholds = True, normalized=True):
    vectors = []
    with open("data_manual_sentiment.json") as json_file:  
        data = json.load(json_file)
        
        for post in data:
            emotions_map = emotion_count.copy()
            if sentiment:
               
                emotions_map["neg"] = 0
                emotions_map["neu"] = 0
                emotions_map["pos"] = 0
                emotions_map["compound"] = 0
            num_sentences = len(post["sentences"])
            for sentence in post["sentences"]:
                if sentiment:
                    vader = sentence["vader"]
                    #print (vader)
                    emotions_map["neg"] += vader["neg"]
                    emotions_map["neu"] += vader["neu"]
                    emotions_map["pos"] += vader["pos"]
                    emotions_map["compound"] += vader["compound"] 
                    
                groups = sentence["jammin"]["groups"]
                for group in groups:
                    if "anger" in group["name"] or "disgust" in group["name"]:
                        for emotion in group["emotions"]:
                            if emotion in emotions_map:
                                emotions_map[emotion] += 1.0

            vector = []
            if normalized:
                for count in emotions_map.values():
                    vector.append(count/float(num_sentences))
            else:
                for count in emotions_map.values():
                    vector.append(count)
                    
            if thresholds:
                vector = vector + get_negativity_ind_posts(post,0,  0, 0)

            vectors.append(vector)
    with open(output,"w") as outfile:  
        json.dump(vectors, outfile)
        outfile.close()
    print(emotions_map)
    return vectors

def some_data_to_vector(indices, output, sentiment = True, normalized=True):
    vectors = []
    with open("data_manual_sentiment.json") as json_file:  
        data = json.load(json_file)
        
        for index in indices:
            post = data[index]
            emotions_map = emotion_count.copy()
            if sentiment:
               
                emotions_map["neg"] = 0
                emotions_map["neu"] = 0
                emotions_map["pos"] = 0
                emotions_map["compound"] = 0
            num_sentences = len(post["sentences"])
            for sentence in post["sentences"]:
                if sentiment:
                    vader = sentence["vader"]
                    #print (vader)
                    emotions_map["neg"] += vader["neg"]
                    emotions_map["neu"] += vader["neu"]
                    emotions_map["pos"] += vader["pos"]
                    emotions_map["compound"] += vader["compound"] 
                    
                groups = sentence["jammin"]["groups"]
                for group in groups:
                    if "anger" in group["name"] or "disgust" in group["name"]:
                        for emotion in group["emotions"]:
                            if emotion in emotions_map:
                                emotions_map[emotion] += 1.0

            vector = []
            for count in emotions_map.values():
                vector.append(count/float(num_sentences))

            vectors.append(vector)
    with open(output,"w") as outfile:  
        json.dump(vectors, outfile)
        outfile.close()
    print(emotions_map)
    return vectors

'''def all_data_to_vector2(output, sentiment=True, emotions=["anger","disgust"], normalized=True):
    vectors = []
    with open("data_manual_sentiment.json") as json_file:  
        data = json.load(json_file)
        
        for post in data:
            emotions_map = emotion_count.copy()
            if sentiment:
                print ("Including sentiment")
                emotions_map["neg"] = 0
                emotions_map["neu"] = 0
                emotions_map["pos"] = 0
                emotions_map["compound"] = 0
            
            num_sentences = len(post["sentences"])
            for sentence in post["sentences"]:
                # Sentiment section
                if sentiment:
                    vader = sentence["vader"]
                    emotions_map["neg"] += vader["neg"]
                    emotions_map["neu"] += vader["neu"]
                    emotions_map["pos"] += vader["pos"]
                    emotions_map["compound"] += vader["compound"]
                    
                
                # Emotion section
                groups = sentence["jammin"]["groups"]
                # For each emotion group
                for group in groups:
                    # For each emotion to include
                    #for emo in emotions:
                        # If the group name matches an emotion to include
                        if "anger" in group["name"]:
                            # If a subemotion within the group is in the emotion map
                            for emotion in group["emotions"]:
                                if emotion in emotions_map:
                                    emotions_map[emotion] += 1.0

            for count in emotions_map.values():
                vector.append(count/num_sentences)

            vectors.append(vector)
            print (vector)
    with open(output,"w") as outfile:  
        json.dump(vectors, outfile)
        outfile.close()
    return vectors
'''

def single_text_to_vector(text, sentiment = True, normalized=True):
    emotions_map = emotion_count.copy()
    if sentiment:
        emotions_map["neg"] = 0
        emotions_map["neu"] = 0
        emotions_map["pos"] = 0
        emotions_map["compound"] = 0
            
    # Split into sentences
    tokenized_sentences = nltk.sent_tokenize(text)
    num_sentences = len(tokenized_sentences)
    for sentence in tokenized_sentences:
        if sentiment:
            vader = analyzer.polarity_scores(sentence)
            #print (vader)
            emotions_map["neg"] += vader["neg"]
            emotions_map["neu"] += vader["neu"]
            emotions_map["pos"] += vader["pos"]
            emotions_map["compound"] += vader["compound"] 
                
        res = jammin_emotion(sentence)
        print (res)
        print()
        groups = res["groups"]
        for group in groups:
            if "anger" in group["name"] or "disgust" in group["name"]:
                for emotion in group["emotions"]:
                    if emotion in emotions_map:
                        emotions_map[emotion] += 1.0

    vector = []
    for count in emotions_map.values():
        vector.append(count/num_sentences)
            
    return [vector]
    
    

        
                            
        
                    
            
            

In [61]:
output = "features_vader_jammin(disgust_anger)_norm.json"
vectors = all_data_to_vector(output, normalized=False)

{'aggravated': 0, 'aggressiveness': 0, 'anger': 1.0, 'anger2': 0, 'angry': 0, 'annoyance': 0, 'annoyed': 0, 'bummed': 0, 'enraged': 0, 'fedup': 1.0, 'frustrated': 0, 'furious': 0, 'grumpy': 1.0, 'ill': 0, 'insulted': 0, 'irritated': 1.0, 'mad': 0, 'offended': 0, 'pissed': 0, 'pissedoff': 0, 'rage': 0, 'bullying': 1.0, 'bitter': 0, 'blah': 0, 'bored': 0, 'boredom': 0, 'cheated': 0, 'contempt': 0, 'defeated': 0, 'disgust': 0, 'disgusted': 0, 'dumb': 0, 'gross': 0, 'horrible': 0, 'hungover': 0, 'lame': 0, 'lousy': 0, 'meh': 0, 'nauseous': 0, 'numb': 0, 'regret': 0, 'remorse': 0, 'rough': 0, 'shame': 0, 'stupid': 0, 'terrible': 0, 'awful': 0, 'ugly': 0, 'yucky': 0, 'neg': 0.124, 'neu': 0.777, 'pos': 0.099, 'compound': -0.1779}


In [62]:
print(len(vectors[0]))
print(vectors[1275])
print(len(vectors))

58
[0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 3.0, 0, 0, 0, 0, 0, 0, 3.0, 1.0, 0, 0, 0, 1.0, 2.0, 0, 2.0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 1.0, 1.761, 3.239, 0.0, -1.3234000000000001, 1.0, 0.35219999999999996, -0.26468, 0.4, 0.4]
1345


In [132]:
single_text_to_vector("You are such a piece of shit")

{'groups': [{'emotions': ['bullying', 'anger', 'furious', 'angry', 'pissedoff'], 'name': 'anger'}, {'emotions': ['awe', 'amazed', 'hyper'], 'name': 'surprise'}], 'text': 'you are such a piece of shit', 'lang': 'en', 'ambiguous': 'no', 'bullying': 'yes'}



[[0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.419,
  0.581,
  0.0,
  -0.5574]]

In [24]:
 print(vectors[9])

[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]


In [ ]:
0: "horrible" 1
1: "ugly" 1
2: "disgusted" 2 
    
0: "meh" 1
1: "blah" 1 
2: "bitter" 1
    

1: "defeated" 1
2: "dumb" 1
3: "terrible" 1
    
0: "grumpy" 1
1: "angry" 1
2: "fedup" 1

## Testing SVM

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [63]:
indices_1 = []
indices_0 = []

with open("data_manual_sentiment.json") as json_file:  
    data = json.load(json_file)
    y = []
    for index, post in enumerate(data):
        if "no_hs" in post["label"]:
            #if len(indices_0) < 330:
            y.append(0)
            indices_0.append(index)
        else:
            y.append(1)
            indices_1.append(index)
print(y.count(0))
print(y.count(1))
print(len(indices_0))
print(len(indices_1))
#indices = indices_0 + indices_1
#output = "./some_features_vader_jammin(disgust_anger)_norm.json"
#vectors = some_data_to_vector(indices, output)

675
670
675
670


In [28]:
print(len(vectors))

1345


In [67]:
#X = [[0], [1], [2], [3]]
#Y = [0, 1, 2, 3]
scoring = ['accuracy','precision', 'recall','f1',
           'precision_micro', 'precision_macro', 'precision_weighted',
           'recall_micro', 'recall_macro', 'recall_weighted',
           'f1_micro','f1_macro','f1_weighted']

c = svm.SVC(kernel='linear', C=1, random_state=0)
#c = svm.SVC(C=1, kernel='linear')

Cs = np.logspace(-6, 3, 10)
grid={"C":Cs, "penalty":["l1","l2"]}# l1 lasso l2 ridge
#grid={"C":Cs}
c=LogisticRegression()
clf = GridSearchCV(estimator=c, param_grid=grid,n_jobs=-1)
clf.fit(vectors, y) 

normalized_X = preprocessing.normalize(vectors)
scores = cross_validate(clf, normalized_X, y, cv=10, scoring=scoring)
print(scores)
print(clf.best_score_)                              
print(clf.best_estimator_.C)




/home/carlos/.virtualenvs/fb/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/carlos/.virtualenvs/fb/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/carlos/.virtualenvs/fb/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/carlos/.virtualenvs/fb/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warn

{'fit_time': array([0.9389174 , 0.80258775, 0.9250493 , 0.69130826, 0.53418493,
       0.6440165 , 0.88817215, 0.70437503, 0.64710307, 0.72693515]), 'score_time': array([0.0095768 , 0.00870538, 0.020576  , 0.00807166, 0.0083065 ,
       0.00783777, 0.00817156, 0.00826168, 0.00805068, 0.00817418]), 'test_accuracy': array([0.5037037 , 0.52592593, 0.51851852, 0.51111111, 0.51851852,
       0.49253731, 0.55970149, 0.55223881, 0.57462687, 0.63432836]), 'train_accuracy': array([0.61404959, 0.60330579, 0.57520661, 0.60578512, 0.6       ,
       0.59620149, 0.61023947, 0.5912469 , 0.60363336, 0.59702725]), 'test_precision': array([0.5       , 0.52459016, 0.51388889, 0.51020408, 0.51851852,
       0.49275362, 0.56896552, 0.56140351, 0.59259259, 0.64516129]), 'train_precision': array([0.62592593, 0.61081081, 0.58348968, 0.61977186, 0.61376673,
       0.60070671, 0.62718447, 0.60588235, 0.61714286, 0.60667904]), 'test_recall': array([0.46268657, 0.47761194, 0.55223881, 0.37313433, 0.41791045,
   

/home/carlos/.virtualenvs/fb/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [162]:
print(scores)

{'fit_time': array([0.78079438, 0.80067778, 0.89862299, 0.73706913, 0.75038624,
       0.73446965, 0.76960468, 0.80942512, 0.76106787, 0.79365849]), 'score_time': array([0.00486231, 0.00490499, 0.00502753, 0.00495887, 0.00533533,
       0.00514221, 0.00509501, 0.00521874, 0.00537539, 0.0054369 ]), 'test_accuracy': array([0.48484848, 0.51515152, 0.45454545, 0.40909091, 0.56060606,
       0.42424242, 0.5       , 0.60606061, 0.54545455, 0.48484848]), 'train_accuracy': array([0.61279461, 0.58922559, 0.6010101 , 0.61279461, 0.56228956,
       0.56902357, 0.58417508, 0.54882155, 0.53198653, 0.54545455]), 'test_precision': array([0.48979592, 0.51282051, 0.45945946, 0.43181818, 0.53571429,
       0.45098039, 0.5       , 0.56363636, 0.52542373, 0.49152542]), 'train_precision': array([0.58354115, 0.57593123, 0.58241758, 0.58481013, 0.53606238,
       0.54485777, 0.56009615, 0.52783109, 0.51711712, 0.52486188]), 'test_recall': array([0.72727273, 0.60606061, 0.51515152, 0.57575758, 0.90909091,
   

## Testing

In [159]:
vector = single_text_to_vector("You are such a piece of shit. I will beat the crap out of you. You have such a horrible face. Eat shit bitch!")
print (vector)

{'groups': [{'emotions': ['bullying', 'pissedoff', 'angry', 'furious', 'anger', 'rage'], 'name': 'anger'}, {'emotions': ['disgusted', 'cheated', 'numb'], 'name': 'disgust'}], 'text': 'you are such a piece of shit.', 'lang': 'en', 'ambiguous': 'no', 'bullying': 'yes'}

{'groups': [{'emotions': ['anger', 'rage', 'bullying', 'furious', 'angry'], 'name': 'anger'}, {'emotions': ['terrible', 'rough', 'bitter'], 'name': 'disgust'}], 'text': 'i will beat the crap out of you.', 'lang': 'en', 'ambiguous': 'no', 'bullying': 'yes'}

{'groups': [{'emotions': ['horrible', 'numb', 'disgusted', 'ugly', 'terrible'], 'name': 'disgust'}, {'emotions': ['grumpy', 'furious', 'anger'], 'name': 'anger'}], 'text': 'you have such a horrible face.', 'lang': 'en', 'ambiguous': 'no', 'bullying': 'no'}

{'groups': [{'emotions': ['bullying', 'pissedoff', 'angry', 'pissed', 'mad', 'fedup', 'ill', 'annoyed'], 'name': 'anger'}], 'text': 'eat shit bitch!', 'lang': 'en', 'ambiguous': 'no', 'bullying': 'yes'}

[[0.0, 0.0,

In [160]:
clf.predict(vector)

array([0])

## Averaging results

In [89]:
from statistics import mean
test_precision_micro = mean([0.48148148, 0.48148148, 0.51851852, 0.54074074, 0.56296296,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])

test_precision_macro = mean([0.48070919, 0.48094816, 0.51837588, 0.54644525, 0.56333333,
       0.49247698, 0.56740443, 0.52457814, 0.61754386, 0.59664306])

test_recall_micro = mean([0.48148148, 0.48148148, 0.51851852, 0.54074074, 0.56296296,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])

test_recall_macro = mean([0.48101405, 0.48112379, 0.51832748, 0.53928885, 0.56255487,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])
    
test_f1_micro = mean([0.48148148, 0.48148148, 0.51851852, 0.54074074, 0.56296296,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])
    
test_f1_macro = mean([0.47916667, 0.48008363, 0.51809545, 0.52160494, 0.56142283,
       0.49151786, 0.56677815, 0.51150604, 0.57861635, 0.56574074])

print("Macro")
print("average precision is %f"%test_precision_macro)
print("average recall is %f"%test_recall_macro)
print("average f1 is %f"%test_f1_macro)

print("Micro")
print("average precision is %f"%test_precision_micro)
print("average recall is %f"%test_recall_micro)
print("average f1 is %f"%test_f1_micro)

Macro
average precision is 0.538846
average recall is 0.534350
average f1 is 0.527453
Micro
average precision is 0.534638
average recall is 0.534638
average f1 is 0.534638


## Prepare data for turkers

In [29]:
with open("data_manual_sentiment.json") as json_file:  
    data = json.load(json_file)
    count = 0
    
    out = open("turk.csv","w")
    out.write("source,sample\n")
    for index, post in enumerate(data):
        if not "no_hs" in post["label"]:
            
            if len(post["text"]) <= 1000:
                count += 1
                text = post["text"] 
                text = text.replace(","," ")
                # Pre process text
                text = text.replace('"','\\"')
                text = text.replace("\n"," ")
                text = text.replace('â€™',"'")
                text = text.replace('â€œ','\\"')
                text = text.replace('â€','\\"')
                text = text.replace('â€”','-')
                text = text.replace('â€¦','...')
                text = text.replace('ðŸ¤”',' ')
                text = text.replace('Ÿ™„',' ')
                text = text.replace('â€˜',"'")
                text = text.replace('ðŸ‘',' ')
                text = text.replace('ðŸ˜‰',' ')
                text = text.replace('ðŸ¤¨',' ')
                text = text.replace('ðŸ˜„',' ')
                text = text.replace('Ÿ˜‚',' ')
                text = text.replace('Ÿ˜',' ')
                text = text.replace('€‹',' ')
                text = text.replace('€‹',' ')
                text = text.replace('„',' ')
                text = text.replace('Ÿ',' ')
                text = text.replace('€',' ')
                text = text.replace('™‚',' ')
                text = text.replace('’',' ')
                text = text.replace('™',' ')
                text = text.replace('‡',' ')
                text = text.replace('‘Š',' ')
                text = text.replace('–',' ')
                text = text.replace('“',' ')
                text = text.replace('”',' ')
                text = text.replace('š',' ')
                text = text.replace('˜',' ')
                text = text.replace('œ…',' ')
                text = text.replace('œ',' ')
                text = text.replace('‹',' ')
                text = text.replace('Œ',' ')
                text = text.replace('›',' ')
                text = text.replace('‘',' ')
                text = text.replace('…',' ')
                text = text.replace('Ž',' ')
                #print(post["text"])
            #else: 
                #print("*"*10+str(index)+"*"*10)
                #print(len(post["text"]))
                out.write(str(index)+ ", \""+ text+"\"\n")
            
    out.close()
    print(count)

658


### Second Additions - After checking paper 2 and noticying it killed us

#### Features extraction

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     
                    ])